In [259]:
import numpy as np
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup as soup
from collections import Counter

In [260]:
df=pd.DataFrame(columns=['Filed','Date','Title','Context'])
base2 = 'https://www.etnews.com'

In [261]:
#target url : 'https://www.etnews.com/news/section/html?id1=20&page='
#과학분야 : https://www.etnews.com/news/section.html?id1=20&id2=020&page=
#에너지분야 : 
for burl in ['https://www.etnews.com/news/section.html?id1=20&id2=020&page=', \
             'https://www.etnews.com/news/section.html?id1=20&id2=065&page=']:
    if 'id2=020' in burl:
        filed='Science'
        print('과학분야 진행중...')
    elif 'id2=065' in burl:
        filed='Energy'
        print('에너지분야 진행중...')
    page = 10000
    url = burl + str(page+1)
    html = urlopen(url)
    bs = soup(html.read(),'html.parser')
    last_page= bs.find('a',{'href':"javascript:;"}).text
    links=[]
    print('링크를 줍는 중...')
    for page in range(int(last_page)):
        url = burl + str(page)
        html = urlopen(url)
        bs = soup(html.read(),'html.parser')
        atc_list = bs.select('p > a')
        for i in range(len(artc_list)):
            if 'https' not in atc_list[i].attrs['href']:
                if atc_list[i].attrs['href'] not in links:
                    links.append(atc_list[i].attrs['href'])
    print('링크에서 기사를 읽는 중...')
    error_links = []
    for link in links:
        try:
            url2 = base2 + link
            html2 = urlopen(url2)
            bs2 = soup(html2.read(),'html.parser')
            date = bs2.find_all('time')[0].text[6:16]
            title = bs2.find('h2').text
            context = bs2.find_all('p')[0].get_text().replace("\n","")
            tmp_df = pd.DataFrame([[filed,date,title,context]],columns=df.columns)
            df = pd.concat([df,tmp_df], ignore_index=True)
        except:
            error_links.append(link)
    print('에러난 링크 재시도중...')
    for link in error_links:
        try:
            url2 = base2 + link
            html2 = urlopen(url2)
            bs2 = soup(html2.read(),'html.parser')
            date = bs2.find_all('time')[0].text[6:16]
            title = bs2.find('h2').text
            context = bs2.find_all('p')[0].get_text().replace("\n","")
            tmp_df = pd.DataFrame([[filed,date,title,context]],columns=df.columns)
            df = pd.concat([df,tmp_df], ignore_index=True)
        except:
            print('로드실패 링크: ',link)
                
    print('중복데이터를 제거하는 중...')
    df = df.loc[~df.duplicated()]

과학분야 진행중...
링크를 줍는 중...
링크에서 기사를 읽는 중...
에러난 링크 재시도중...
중복데이터를 제거하는 중...
에너지분야 진행중...
링크를 줍는 중...
링크에서 기사를 읽는 중...
에러난 링크 재시도중...
중복데이터를 제거하는 중...


In [258]:
df.duplicated().sum()

0

In [256]:
df.shape

(1514, 4)

In [255]:
df.to_csv("etnews.csv")